In [1]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
from platform import python_version

print(python_version())
import os
os.environ['CURL_CA_BUNDLE'] = ''
model_name = ['all-MiniLM-L6-v2', 'stsb-roberta-large', 'stsb-mpnet-base-v2', 'allenai-specter']

with open("DFG.txt") as f:
    dfg = f.read()
dfg_q = dfg.split("\n")
dfg_q = [q for q in dfg_q if q != "" and q != " "]
with open ("Horizon_Europe.txt") as f:
    h_e = f.read()
h_e_q = h_e.split("\n")
h_e_q = [q for q in h_e_q if q != "" and q != " "]

3.9.16


In [2]:
from transformers import AutoTokenizer, AutoModel

# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/specter2')

#load base model
model = AutoModel.from_pretrained('allenai/specter2')

#load the adapter(s) as per the required task, provide an identifier for the adapter in load_as argument and activate it
model.load_adapter("allenai/specter2_adhoc_query", source="hf", load_as="adhoc_query", set_active=True)

papers = [{'title': 'BERT', 'abstract': 'We introduce a new language representation model called BERT'},
          {'title': 'Attention is all you need', 'abstract': ' The dominant sequence transduction models are based on complex recurrent or convolutional neural networks'}]

# concatenate title and abstract
text_batch = [d['title'] + tokenizer.sep_token + (d.get('abstract') or '') for d in papers]
# preprocess the input
inputs = self.tokenizer(text_batch, padding=True, truncation=True,
                                   return_tensors="pt", return_token_type_ids=False, max_length=512)
output = model(**inputs)
# take the first token in the batch as the embedding
embeddings = output.last_hidden_state[:, 0, :]

AttributeError: 'BertModel' object has no attribute 'load_adapter'

In [6]:
import pandas as pd
df1 = pd.DataFrame(
    {
    # "DFG number" : list(map(lambda x: x+1, list2[0])),
    "DFG question content" : dfg_q,
    }


    )

for i,name in enumerate(model_name):
    model = SentenceTransformer(name)
    embedding_dfg = model.encode(dfg_q, convert_to_tensor=True)
    embedding_he = model.encode(h_e_q, convert_to_tensor=True)
    # compute similarity scores of two embeddings
    cosine_scores = util.pytorch_cos_sim( embedding_dfg, embedding_he )
    top_k=2
    result_list = []
    list3 = [[],[],[],[],[],[],[],[]]
    q2 = h_e_q
    q1 = dfg_q
    for i, sentence in enumerate(q1):

        top_results = np.argpartition(-cosine_scores.cpu()[i], range(top_k))[0:top_k]
        result_list.append([i,int(top_results[0]) ])
        list3[0].append(i)
        list3[1].append(int(top_results[0]))
        list3[2].append(float(max(list(cosine_scores[i]))))
        list3[3].append(sentence)
        list3[4].append(q2[top_results[0]])
        list3[5].append(q2[top_results[1]])
        list3[6].append(int(top_results[1]))
        list3[7].append(float(cosine_scores[i][int(top_results[1])]))

    #     result_list.append([i, int(top_results[1]) ])
    #     print("DFG Question:", sentence, "")
    #     print("Top", top_k, "most similar sentences in DFG checklist:")
    #     for idx in top_results[0:top_k]:
    #         print(q2[idx], "Score:{:.4f}, No. of questions:{}".format(cosine_scores[i][idx],idx+1 ))

    #     print("\n")

    df1[name+"_1" ] = list3[1]
    df1[name+"_2" ] = list3[6]
df1.to_csv("dfg_similar_questions_all_model.csv")

In [7]:
# horizon europe as 
import pandas as pd
df1 = pd.DataFrame(
    {
    # "DFG number" : list(map(lambda x: x+1, list2[0])),
    "Horizon Europe question content" : h_e_q,
    }


    )

for i,name in enumerate(model_name):
    model = SentenceTransformer(name)
    embedding_dfg = model.encode(dfg_q, convert_to_tensor=True)
    embedding_he = model.encode(h_e_q, convert_to_tensor=True)
    # compute similarity scores of two embeddings
    cosine_scores = util.pytorch_cos_sim( embedding_he, embedding_dfg )
    top_k=2
    result_list = []
    list3 = [[],[],[],[],[],[],[],[]]
    q1 = h_e_q
    q2 = dfg_q
    for i, sentence in enumerate(q1):

        top_results = np.argpartition(-cosine_scores.cpu()[i], range(top_k))[0:top_k]
        result_list.append([i,int(top_results[0]) ])
        list3[0].append(i)
        list3[1].append(int(top_results[0]))
        list3[2].append(float(max(list(cosine_scores[i]))))
        list3[3].append(sentence)
        list3[4].append(q2[top_results[0]])
        list3[5].append(q2[top_results[1]])
        list3[6].append(int(top_results[1]))
        list3[7].append(float(cosine_scores[i][int(top_results[1])]))

    #     result_list.append([i, int(top_results[1]) ])
    #     print("DFG Question:", sentence, "")
    #     print("Top", top_k, "most similar sentences in DFG checklist:")
    #     for idx in top_results[0:top_k]:
    #         print(q2[idx], "Score:{:.4f}, No. of questions:{}".format(cosine_scores[i][idx],idx+1 ))

    #     print("\n")

    df1[name+"_1" ] = list3[1]
    df1[name+"_2" ] = list3[6]
df1.to_csv("he_similar_questions_all_model.csv")